In [ ]:
import numpy as np
import math
import re

In [ ]:
!wget http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz

--2021-09-17 22:21:38--  http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Electronics_5.json.gz
Resolving snap.stanford.edu (snap.stanford.edu)... 171.64.75.80
Connecting to snap.stanford.edu (snap.stanford.edu)|171.64.75.80|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 495854086 (473M) [application/x-gzip]
Saving to: ‘reviews_Electronics_5.json.gz.1’

reviews_Electronics 100%[===================>] 472.88M  3.54MB/s    in 2m 41s  

2021-09-17 22:24:20 (2.94 MB/s) - ‘reviews_Electronics_5.json.gz.1’ saved [495854086/495854086]



In [ ]:
# Reading the data
import json
import gzip

json_content = []
file = 'reviews_Electronics_5.json.gz'
with gzip.open(file , 'rb') as gzip_file:
    for line in gzip_file:

        line = line.rstrip()  # removing xtra spaces
        if line:
            obj = json.loads(line)
            json_content.append(obj['reviewText'])
            
            if len(json_content) == 50000:
                break

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# sentences = 1D array of sentence
from nltk.tokenize import sent_tokenize, word_tokenize

sentences = []
for i in json_content:
    sentences += sent_tokenize(i)

print(sentences[0])
# print(word_tokenize(sentences[1][0]))

We got this GPS for my husband who is an (OTR) over the road trucker.


In [ ]:
# removing punctuation
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

# words = 1D array of words
words = []
for i in sentences:
    words.append("<S>")
    words += tokenizer.tokenize(i.lower())  # lowercase words
    # words.append("<E>")

print(words[:25])

['<S>', 'we', 'got', 'this', 'gps', 'for', 'my', 'husband', 'who', 'is', 'an', 'otr', 'over', 'the', 'road', 'trucker', '<S>', 'very', 'impressed', 'with', 'the', 'shipping', 'time', 'it', 'arrived']


In [ ]:
from collections import defaultdict

freq = defaultdict(int)
for i in words:
  freq[i]+=1

# removing words freq < 5
new_words = [i for i in words if freq[i] > 5]

print(new_words[:25])

['<S>', 'we', 'got', 'this', 'gps', 'for', 'my', 'husband', 'who', 'is', 'an', 'over', 'the', 'road', 'trucker', '<S>', 'very', 'impressed', 'with', 'the', 'shipping', 'time', 'it', 'arrived', 'a']


In [ ]:
# Generating training data

# word2ind & ind2word
word2ind=defaultdict(list)
ind2word=defaultdict(list)

c = 0
# giving each word its index
for i in freq:
  if freq[i] > 5:
    ind2word[c]=i
    word2ind[i]=c
    c+=1

# print(freq[new_words[word2ind["<E>"]]])
# print(new_words[word2ind["<E>"]])

In [ ]:
import pickle

# 1. storing ind2word
# embedding = [i for i in ind2word]
file = open("./drive/MyDrive/NLP/q2/ind2word.pkl","wb")
i2w = { i:ind2word[i] for i in ind2word}
pickle.dump(i2w,file)
file.close()

# 2. storing word2ind
# embedding = [i for i in word2ind]
file = open("./drive/MyDrive/NLP/q2/word2ind.pkl","wb")
w2i = { i:word2ind[i] for i in word2ind}
pickle.dump(w2i,file)
file.close()

# print(word2ind)

In [ ]:
# Generating training data

# We can uniquely give one hot encoding to the vectors by using their index nos (Word2Ind)
# def one_hot_encode(word, V):
#   vec = np.zeros(V)
#   vec[word2ind[word]] = 1
#   return vec

def softmax(x):
    e_x = np.exp(x)
    ycap = e_x/np.sum(e_x, axis=0)
    return ycap

In [ ]:
# Initializing weight matrices W1 & W2
def init_model(N,V):
  W1 = np.random.rand(V,N)
  W2 = np.random.rand(N,V)
  return W1,W2

# forward propogation
def forward_prop(x, W1, W2):
  h=np.dot(x,W1)
  u=np.dot(h,W2) 
  y_c = softmax(u.T)
  return y_c.T, h, u

# backward propogation
def back_prop(x, h, err, batch_size, W2):
    gradW1 = np.dot(x.T, np.dot(err, W2.T) ) / batch_size
    gradW2 = np.dot(h.T, err) / batch_size
    return gradW1, gradW2

epoch = -1
def get_vectors(batch_size, window):
  global epoch
  while True:
    epoch+=1
    for i in range(0,len(new_words)):
      if new_words[i] == "<S>": continue
      
      w_context=np.zeros(V)
      w_target=np.zeros(V)
      w_target[word2ind[new_words[i]]]=1
      
      mean=0
      # back loop
      for j in range(1,window+1):
        if i-j<0 or new_words[i-j] == "<S>": break
        w_context[word2ind[new_words[i-j]]]+=1
        mean+=1

      # forward loop
      for j in range(1,window+1):
        if i+j>=len(new_words) or new_words[i+j] == "<S>": break
        w_context[word2ind[new_words[i+j]]]+=1
        mean+=1        
      
      if mean:
        w_context/=mean   
      yield w_context,w_target

def get_batches(batch_size, window):
  batch_target=[]
  batch_context=[]
  for x,y in get_vectors(batch_size, window):
    if len(batch_context) < batch_size:
      batch_context.append(x)
      batch_target.append(y)
    else:
      yield np.array(batch_context),np.array(batch_target)
      batch_target=[]
      batch_context=[]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# save embeddings periodically
import pickle

def save_embeddings(W1, W2, ycap, y, h, iterations, eta):
  embedding = [i for i in W2]
  file = open("./drive/MyDrive/NLP/w2.pkl","wb")
  pickle.dump(embedding,file)
  file.close()

  embedding = [i for i in W1]
  file = open("./drive/MyDrive/NLP/w1.pkl","wb")
  pickle.dump(embedding,file)
  file.close()

  embedding = ycap
  file = open("./drive/MyDrive/NLP/ycap.pkl","wb")
  pickle.dump(embedding,file)
  file.close()

  embedding = y
  file = open("./drive/MyDrive/NLP/y.pkl","wb")
  pickle.dump(embedding,file)
  file.close()

  embedding = h
  file = open("./drive/MyDrive/NLP/h.pkl","wb")
  pickle.dump(embedding,file)
  file.close()

  embedding = iterations
  file = open("./drive/MyDrive/NLP/iterations.pkl","wb")
  pickle.dump(embedding,file)
  file.close()

  embedding = eta
  file = open("./drive/MyDrive/NLP/eta.pkl","wb")
  pickle.dump(embedding,file)
  file.close()

In [ ]:
# Training

# mini batch gradient descent
def gradient_descent(batch_size, eta, iterations, N, V, window):
  W1,W2 = init_model(N,V)
  
  for x,y in get_batches(batch_size, window):
    iterations-=1

    # forward propogation
    ycap, h, u = forward_prop(x, W1, W2)

    # back propagation
    err = ycap - y
    gradientW1, gradientW2 = back_prop(x, h, err, batch_size, W2)

    W1 -= eta*gradientW1
    W2 -= eta*gradientW2

    if iterations%100 == 0:
      eta *= 0.7
    if iterations%1000 == 0:
      save_embeddings(W1, W2, ycap, y, h, iterations, eta)
    if iterations <= 0:
      break
  
  return W1, W2


In [ ]:
# parameters
batch_size = 64
eta = 0.035 # rate of change (η)
N = 100
iterations = 150000
window = 2
V = len(word2ind) # Unique Vocabulary

# calling gradient descent
W1, W2 = gradient_descent(batch_size, eta, iterations, N, V, window)

# print(W1)
# print(W2)

In [ ]:
# returns top10 nearest neighbours to 'str' string
def top10(str):
  array = []
  v1 = W2.T[word2ind[str]]
  for i in range (0,len(W2.T)):
    # angle = cosine_similarity(v1, U[i])
    v2 = W2.T[i]
    angle = np.dot(v1,v2)/(np.linalg.norm(v1)*np.linalg.norm(v2))
    array.append([angle,ind2word[i]])
  
  array.sort(reverse=True)
  # print(array[:10])
  return array[:10]

print(top10("camera"))

In [ ]:
print(top10("awesome"))

In [ ]:
print(top10("terrible"))

In [ ]:
print(top10("device"))

In [ ]:
print(top10("good"))

In [ ]:
print(top10("hard"))